In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

In [3]:
#个人写的脚本文件
#更新数据库信息
from update_sql import update_sql,info_sql
#画K线图
from plot_stock import stock_plot

In [190]:
#更新数据库
update_sql()

数据库已更新至20200113日数据


In [191]:
#获取数据库信息
info_sql()

统计查询的总数：7873752
数据期间：20050104——20200113
数据库包含股票个数：3760


## 寻找价量突破个股

In [196]:
#个人写的脚本文件
from base import sql_engine,ts_pro
from datetime import datetime,timedelta
pro=ts_pro()
engine = sql_engine()

#从数据库中获取复权价格和成交量
def get_price_vol_data():
    now=datetime.now()
    date=(now-timedelta(360)).strftime('%Y%m%d')
    sql=f'select * from daily_data where trade_date>{date}'
    all_data=pd.read_sql(sql,engine)
    all_data=all_data.sort_values(['ts_code','trade_date'])
    codes=list(all_data.ts_code.unique())
    #前复权
    all_data['adjclose']=all_data.groupby('ts_code').apply(lambda x:x.close*x.adj_factor/x.adj_factor.iloc[-1]).values
    all_data['adjvol']=all_data.groupby('ts_code').apply(lambda x:x.vol*x.adj_factor/x.adj_factor.iloc[-1]).values
    all_data['adjopen']=all_data.groupby('ts_code').apply(lambda x:x.open*x.adj_factor/x.adj_factor.iloc[-1]).values
    all_data['adjhigh']=all_data.groupby('ts_code').apply(lambda x:x.high*x.adj_factor/x.adj_factor.iloc[-1]).values
    all_data['adjlow']=all_data.groupby('ts_code').apply(lambda x:x.low*x.adj_factor/x.adj_factor.iloc[-1]).values
    
    #设置索引
    all_data=all_data.set_index(['trade_date','ts_code'])[['adjclose','adjvol','adjopen','adjhigh','adjlow']]
    #转成面板数据
    all_data=all_data.unstack()
    return codes,all_data

#筛选价格和成交量突破N日阈值的个股
def find_price_vol_stock(n,r=1.2):
    codes,all_data=get_price_vol_data()
    up_list=[]
    for code in codes:
        close=all_data['adjclose'][code]
        open_=all_data['adjopen'][code]
        high=all_data['adjhigh'][code]
        low=all_data['adjlow'][code]
        vol=all_data['adjvol'][code]
        #剔除一字涨停
        flag=True
        if close.iloc[-1]==open_.iloc[-1]==high.iloc[-1]==low.iloc[-1]:
            flag=False
            break
        #最近五日没有长上影线,以单日回撤3%为长上影线
        for i in range(5):
            if close[-5:][i]*1.03<high[-5:][i]:
                flag=False
                break
                
        #价格突破前N日新高
        p=close.iloc[-1] #当前价格
        p0=close[-n:-1].min()
        p1=close[-n:-1].max() #前n-1日最高价
        
        #价格短期已上涨超过50%，涨幅过大不宜介入
        '''
        if (p-p0)/p0>r:
            flag=False
            break '''
        #价格突破且放量上涨
        if flag==True and \
           p1<p<p1*r and \
           vol[-5:].mean()/vol[-10:-5].mean()>2.0:
            up_list.append(code)
    return up_list

In [193]:
stocks_20=find_price_vol_stock(20)
print('突破20日量价的个股为：\n')
print(stocks_20)
print(f'突破20日量价个股个数为：{len(stocks_20)}')

突破20日量价的个股为：

['000417.SZ', '000885.SZ']
突破20日量价个股个数为：2


In [199]:
stock_plot(stocks_20[0]).kline_plot(ktype=0)

In [195]:
stock_plot(stocks_20[1]).kline_plot(ktype=0)

## 寻找价格突破N日W底/圆形底个股

In [202]:
from RPS import get_data
data=get_data()
data.tail()

ts_code,雅克科技,维科技术,翠微股份,正丹股份,吉华集团,卧龙电驱,远东传动,苏美达,江苏有线,大唐电信,...,通鼎互联,华鹏飞,汇金科技,中公高科,深赛格,中国太保,东富龙,中国应急,环旭电子,宜华生活
trade_date,,,,,,,,,,,,,,,,,,,,,
20200107,25.75,7.10,7.27,5.12,8.31,12.56,6.07,5.66,4.15,11.09,...,6.97,5.53,16.54,27.66,7.87,39.09,7.37,11.45,20.88,3.09
20200108,25.01,7.09,7.06,5.06,8.19,12.79,6.09,5.76,4.12,10.70,...,6.82,5.37,15.68,28.31,7.51,38.50,7.24,12.60,20.18,3.02
20200109,25.97,7.10,7.14,5.17,8.27,12.86,6.38,5.81,4.14,10.87,...,7.00,5.46,16.79,28.19,7.62,38.90,7.36,12.23,20.54,3.14
20200110,25.71,6.84,7.24,5.23,8.41,12.44,6.21,5.77,4.07,10.87,...,6.98,5.40,16.54,27.75,8.38,38.45,7.46,11.96,20.17,3.08
20200113,27.13,6.87,7.26,5.26,8.39,12.77,6.19,5.76,4.10,10.94,...,7.00,5.38,16.71,27.80,8.25,38.81,7.48,11.89,20.70,3.08


In [203]:
def find_stock(data,n=20):
    stock_list=[]
    for c in data.columns:
        d0=data[c][-n]
        d1=data[c][-(n-2):-1].max()
        d2=data[c][-1]
        if 3<d2<20 and d1<d0<d2<d0*1.52:
            stock_list.append(c)
    #print(len(stock_list))
    return stock_list

In [204]:
ss_20=find_stock(data)
print(ss_20)

['立霸股份', '恒华科技', '普洛药业', '鼎信通讯', '东方电缆']


In [205]:
ss_60=find_stock(data,n=60)
print(ss_60)

['鲁阳节能', '北汽蓝谷', '农尚环境', '中航重机', '金牛化工']


In [206]:
stock_plot(ss_20[0]).kline_plot(ktype=0)

In [207]:
stock_plot(ss_20[1]).kline_plot(ktype=0)

In [253]:
stock_plot(ss_20[2]).kline_plot(ktype=0)

In [208]:
stock_plot(ss_60[0]).kline_plot(ktype=0)

In [209]:
stock_plot(ss_60[1]).kline_plot(ktype=0)

In [210]:
stock_plot(ss_60[2]).kline_plot(ktype=0)